# 对乐谱进行变换

In [1]:
import numpy as np
import re 

## 数字音高与音名的转换

In [2]:
note_num={"C":0, "Db":1, "D":2, "Eb":3,"E":4, "F":5, "Gb":6,
"G":7, "Ab":8, "A":9, "Bb":10, "B":11,}
num_note={v:k for k,v in note_num.items()}

In [3]:
# Conversion of digital pitch and sound names

def pitch_to_name(pitch, root):
    root_num=note_num[root]+60 
    pitch_num=pitch-root_num
    octave=pitch_num//12+4
    pitch_num=pitch_num%12
    return num_note[pitch_num], octave

def name_to_pitch(name, octave, root):
    root_num=note_num[root]+60
    pitch_num=note_num[name]+12*(octave-4)
    return pitch_num+root_num


## Scale的重映射

对于major/Ionian：

* 音高是
  * [C,D,E,F,G,A,B]
  * [0,2,4,5,7,9,11]
   
对于minor/Aeolian 
* 音高是
  * [C,D,Eb,F,G,Ab,Bb]
  * [0,2,3, 5,7,8,10]

重映射时：

|原scale|原音高|新scale|新音高|
|:--|:--|:--|:--|
|E|4|Eb|3|
|A|9|Ab|8|
|B|11|Bb|10|

整体：

|序数|0|-|1|-|2|3|-|4|-|5|-|6|
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
|Ionian|0|1|2|3|4|5|6|7|8|9|10|11|
|Aeolian|0|1|2|-|3|5|6|7|-|8|-|10|

|序数  | 0|   1|  2|  3|  4|  5|  6|
|:--   |:--|:--|:--|:--|:--|:--|:--|
|Ionian|        C|  D|  E|  F|  G|  A|  B|
|Aeolian|       C|  D| Eb|  F|  G| Ab| Bb|
|Dorian|        C|  D| Eb|  F|  G|  A| Bb|
|Phrygian|      C| Db| Eb|  F|  G| Ab| Bb|
|Lydian|        C|  D|  E| Gb|  G|  A|  B|
|Mixolydian|    C|  D|  E|  F|  G|  A| Bb|
|Locrian|       C| Db| Eb|  F| Gb| Ab| Bb|

In [4]:
Ionian = ["C","D","E","F","G","A","B"]
Aeolian = ["C","D","Eb","F","G","Ab","Bb"]
Dorian = ["C","D","Eb","F","G","A","Bb"]
Phrygian = ["C","Db","Eb","F","G","Ab","Bb"]
Lydian = ["C","D","E","Gb","G","A","B"]
Mixolydian = ["C","D","E","F","G","A","Bb"]
Locrian = ["C","Db","Eb","F","Gb","Ab","Bb"]
Harmonic_Minor = ["C","D","Eb","F","G","Ab","B"]
Blues=["C","Db","Eb","F","Gb","G","Bb"]

In [5]:
def note_to_index(note, scale):
    scale_num = np.asarray([note_num[x] for x in scale])
    if note in scale:
        return scale.index(note)
    else:
        x=np.count_nonzero(note_num[note] > np.array(scale_num))-0.5 
        return x 

In [6]:
def trans_scale(note, scale_origin, scale_target):
    origin_note_index= note_to_index(note, scale_origin)
    if int(origin_note_index) == origin_note_index:
        target_note=scale_target[origin_note_index]
    else:
        # 如果序数不为整数，取上下两个音符的平均值
        lower_note_index=int(origin_note_index)
        lower_note=scale_target[lower_note_index]
        upper_note_index=int(origin_note_index)+1
        upper_note=scale_target[upper_note_index]
        target_note_num=np.round((note_num[lower_note]+note_num[upper_note])/2)
        target_note=num_note[target_note_num]
    return target_note

## 遍历所有音符

```
<step>x1</step><octave>y1</octave> -> m
m ->变调式 -> n
<step>x2</step><octave>y2</octave>
```

In [7]:
def num_to_step_octave(pitch, root):
    name, octave = pitch_to_name(pitch, root)
    find_str = f"<step>{name}</step>[\s\n]*<octave>{octave}</octave>"
    return find_str

def trans_note_round_A(pitch, xml_str,  
                origin_scale, origin_root, 
                target_scale, target_root):
    origin_name, origin_octave = pitch_to_name(pitch, origin_root)
    find_str = f"<step>{origin_name}</step>[\s\n]*<octave>{origin_octave}</octave>"
    find_str= re.compile(find_str)
    target_name=trans_scale(origin_name,origin_scale, target_scale)
    target_pitch=name_to_pitch(target_name,origin_octave,target_root)
    tmp_str=f"<tmp>{target_pitch}</tmp>"
    middle_xml_str=re.sub(find_str, tmp_str, xml_str)
    return middle_xml_str

def trans_note_round_B(pitch, xml_str,  
                origin_scale, origin_root, 
                target_scale, target_root):
    target_name, target_octave= pitch_to_name(pitch, target_root) 
    tmp_str=re.compile(f"<tmp>{pitch}</tmp>")
    replace_str= f"<step>{target_name}</step><octave>{target_octave}</octave>"
    return re.sub(tmp_str, replace_str, xml_str)


In [8]:
def double_round(xml_str,  
                origin_scale, origin_root, 
                target_scale, target_root):
    for p in range(1,127):
        middle_str = trans_note_round_A(p, xml_str,  
                    origin_scale, origin_root, 
                    target_scale,target_root)
        xml_str=middle_str
    for p in range(1,127):
        middle_str = trans_note_round_B(p, xml_str,  
                    origin_scale, origin_root, 
                    target_scale,target_root)
        xml_str=middle_str
    return xml_str


In [9]:
with open('Canon_in_C.musicxml', 'r') as f:
    xml_str = f.read()
    new_xml_str=double_round(xml_str,Ionian,"D",Aeolian,"D")

with open('Canon_in_C_new.musicxml', 'w') as f:
    f.write(new_xml_str)